In [10]:
from scipy.optimize import nnls
from functools import partial

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ast
import torch
from sklearn.decomposition import NMF

from random import sample

from functions import cosine_perm, simulate_mixedBIG, simulate_mixedLittle, simulate_counts
from AAUtoSig_init import AAUtoSig, train_AAUtoSig

## Data management
Load the frequency matrix generated by the tool SomaticSiMu, and the set of signatures used for generating each instance. These are then matches by the patient ID, to determine what signatures are present in a given sub-sample

In [11]:
m = pd.read_csv(r'Q:\AUH-HAEM-FORSK-MutSigDLBCL222\article_1\generated_data\SomaticSiMu\Lung-AdenoCA_SBS96.csv')#.drop(['Unnamed: 0'], axis=1)

In [12]:
groundtruth = [line.strip("\n") for line in open(r'Q:\AUH-HAEM-FORSK-MutSigDLBCL222\article_1\generated_data\SomaticSiMu\Lung-AdenoCA_sbs_sigs.txt')]
groundtruth = pd.DataFrame([ast.literal_eval(x) for x in groundtruth])
groundtruth[0] = groundtruth[0].astype(int)


sig_names = np.unique(groundtruth.drop([0], axis = 1).fillna(":("))[1::]
COSMIC = pd.read_csv(r'COSMIC\COSMIC_v3.2_SBS_GRCh37.txt', sep = '\t', index_col=0)
sigs = COSMIC[sig_names]

In [13]:
xd = groundtruth[0].sort_values().tolist()
for i in range(len(xd)-1):
    if not (xd[i] + 1 == xd[i + 1]):
        print(xd[i])


25
33
161
361
533
565
597
629
661
692
713
721


## Split data
Firstly we write a method to split the mutational profile matrix into a training and validation set.

In [14]:
def split_data(data, frac_1):
  x_train = data.sample(frac = frac_1)
  x_validation = data.drop(x_train.index)
  return x_train, x_validation

In [15]:
train_m, test_m = split_data(m.T, 0.8)

x = train_m.index
y = [str(x) for x in groundtruth[0]]

ok  = list (set(x) & set(y))

train_m[train_m.index.isin(ok)]



,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
346,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
446,31,17,12,18,49,45,13,43,31,27,...,6,1,2,1,4,2,1,1,3,6
217,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
88,3,4,1,2,4,1,2,4,8,2,...,2,4,1,1,1,1,1,0,1,2
246,32,19,8,17,47,48,11,28,18,17,...,2,2,2,1,4,2,2,2,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,1,0,0,3,0,2,0,1,1,0,...,0,0,0,0,0,2,1,1,0,0
239,4,2,0,3,10,7,3,3,5,3,...,1,0,0,0,5,0,0,0,0,1
479,0,0,0,1,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
229,9,6,1,3,6,8,1,6,5,3,...,1,3,0,1,2,4,1,2,0,8


# NMF performance

In [16]:
def out_errorNMF(train_df, validation_df, nsigs, true_sigs):
  model = NMF(n_components=nsigs, init='random', max_iter = 500)
  exposures = model.fit_transform(train_df)
  signatures = model.components_
  cos_NMF = cosine_perm(true_sigs.to_numpy().T, signatures)[0]
  cos_mean = np.mean(cos_NMF.diagonal())


  ref_exposures = model.transform(X = validation_df)
  rec = np.dot(ref_exposures, signatures)
  MSE = np.mean(((validation_df - rec)**2).to_numpy())
  
  #return(MSE)
  return([cos_mean, MSE])

# AAUtoSig performance

In [17]:
def out_error_AAUtoSig(train_df, validation_df, nsigs,true_sigs):
    model = AAUtoSig(nsigs)

    # Validation using MSE Loss function
    loss = torch.nn.MSELoss(reduction='mean')

     # Using an Adam Optimizer with lr = 1e-3
    optimizer = torch.optim.Adam(model.parameters(),
                            lr = 1e-3)
    train_AAUtoSig(500, model, train_df, loss_function= loss, optimizer=optimizer, batch_size= 32)

    signatures = model.dec1.weight.data    
    signatures = pd.DataFrame(signatures.numpy())
    asd = cosine_perm(true_sigs.to_numpy().T, signatures.T)
    cos_Deep = asd[0]
    cos_mean = np.mean(cos_Deep.diagonal())

    def calculate_exp(b):
       return(nnls(A = signatures, b = b)[0])
   
    ref_exposures = np.stack(validation_df.apply(calculate_exp, axis = 1))
    rec = np.dot(ref_exposures, signatures.T)

    MSE = np.mean(((validation_df - rec)**2).to_numpy())
    #return(MSE)
    return ([cos_mean, MSE])

## Performance analysis

In [18]:
def performance_analysis(m, m_sigs):
  mut_matrix = (m.drop(['Unnamed: 0', '0'], axis = 1)).T.sample(400)
  train_data, test_data = split_data(mut_matrix, 0.75)
  
  idx_train = train_data.index
  idx_test = test_data.index
  y = [str(x) for x in m_sigs[0]]

  train_idx = list (set(idx_train) & set(y))
  test_idx = list(set(idx_test) & set(y))
  train_data = train_data[train_data.index.isin(train_idx)]
  test_data = test_data[test_data.index.isin(test_idx)]

  m_sigs_train = m_sigs[m_sigs[0].isin([int(x) for x in train_idx])]
  m_sigs_test = m_sigs[m_sigs[0].isin([int(x) for x in test_idx])]
  sig_names_train = np.unique(m_sigs_train.drop([0], axis = 1).fillna(":("))[1::]
  sig_names_test = np.unique(m_sigs_test.drop([0], axis = 1).fillna(":("))[1::]

  COSMIC = pd.read_csv(r'COSMIC\COSMIC_v3.2_SBS_GRCh37.txt', sep = '\t', index_col=0)
  sigs_train = COSMIC[sig_names_train]
  nsigs = len(sig_names_train)

  print(sig_names_train)
  print(sig_names_test)
  return(out_errorNMF(train_data, test_data, nsigs, sigs_train) + 
            out_error_AAUtoSig(train_data, test_data, nsigs, sigs_train))

asd = [performance_analysis(m, groundtruth) for _ in range(1)] 
result = pd.DataFrame(asd)
result.columns = ["cosineNMF linear", "outNMF linear", "cosineAE linear", "outAE linear"]


['SBS1' 'SBS13' 'SBS17a' 'SBS17b' 'SBS18' 'SBS2' 'SBS28' 'SBS3' 'SBS4'
 'SBS40' 'SBS5' 'SBS9']
['SBS1' 'SBS13' 'SBS17a' 'SBS17b' 'SBS18' 'SBS2' 'SBS3' 'SBS4' 'SBS40'
 'SBS5']


C:\Users\bjyw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
plt.boxplot(result[['outNMF linear', 'outAE linear']], labels = ["NMF", "AE"])
plt.title('NMF vs AE out of sample error on linearly simulated data')
plt.show()

plt.boxplot(result[['cosineNMF linear', 'cosineAE linear']] ,labels = ["NMF", "AE"])
plt.title('NMF vs AE cosine values on linearly simulated data')
plt.show()

plt.scatter(y = result['cosineNMF linear'], x = result['outNMF linear'], c = 'blue', label = 'NMF')
plt.scatter(y = result['cosineAE linear'], x = result['outAE linear'], c = 'red', label = 'AE')
plt.xlabel('Out of sample error')
plt.ylabel('mean diagonal cosine')
plt.title('NMF vs AE performance on linearly simulated data')
plt.legend()
plt.show()
#plt.savefig('linear.png', transparent=True)